<a href="https://colab.research.google.com/github/HugoTHO/sisbi-utilities-notebooks/blob/main/sigaa_register_daily_movement_of_users.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AUTOMAÇÃO DE REGISTRO DE MOVIMENTAÇÃO DIÁRIA DE USUÁRIOS
***

## Importação de bibliotecas Python

In [52]:
import bs4
import requests
import urllib.parse as urlparser

from getpass import getpass
from ipywidgets import widgets
from html import unescape

## Login

In [ ]:
#@markdown Execute esta célula e forneça sua credencial do Sigaa e senha. <br>
#@markdown Em seguida, escolha seu vínculo. <hr>
#@markdown O login no sistema e escolha do vínculo são necessários para as 
#@markdown próximas operações.

base_url = "https://sigaa.ufrn.br"
base_path = "/sigaa/"
relship_sel_path = "/sigaa/vinculos.jsf"

# INTERFACE UTILITIES
dropbox_placeholder = "SELECIONE O VINCULO"
default_box_lt = widgets.Layout(padding='0 0 28px 0', border='1px solid white')
success_box_lt = widgets.Layout(padding='0 0 28px 0', border='1px solid green')

session = requests.Session()
login_page = session.get(urlparser.urljoin(base_url, base_path))

username = input("Usuário: ")
password = getpass("Senha: ")

# Does mount the POST necessary login data
login_form = (
    bs4.BeautifulSoup(login_page.text)
    .find('form', {'id':'login-form'})
)
login_path = login_form['action']
login_data = {
    'username':username,
    'password':password
}
for hidden_input in login_form.find_all('input',{'type':'hidden'}):
    login_data[hidden_input['name']] = hidden_input['value']

response = session.post(
    urlparser.urljoin(login_page.url,login_path),
    data=login_data
)

if len(response.history) and response.history[0].status_code > 300:
    print('Login realizado com sucesso, escolha seu vínculo:')

    relationship_page = session.get(
        urlparser.urljoin(base_url, relship_sel_path)
    )
    relship_links = (
        bs4.BeautifulSoup(relationship_page.text)
        .find_all('a', {'class': 'withoutFormat'})
    )

    # Does concatenate the relationship type and description on link list,
    # converting the escaped charactes.
    choices = [
        (reltype.string + 
        unescape(relship.string.split(':')[1]
            .encode('ascii', 'xmlcharrefreplace')
            .decode('utf-8'))
        )
        for reltype, relship in zip(
            relship_links[::3],
            relship_links[2::3]
        )
    ]

    relship_dd = widgets.Dropdown(
        options = [dropbox_placeholder] + choices,
        value = dropbox_placeholder,
        layout = default_box_lt
    )
    display(relship_dd)

    # Does register a handler function to access the choiced relationship
    def dd_change(change):
        if (change['new'] != 0):
            relship_path = relship_links[(change['new']-1)*3]['href']
            session.get(urlparser.urljoin(base_url, relship_path))
            relship_dd.layout = success_box_lt
        else:
            relship_dd.layout = default_box_lt

    relship_dd.observe(dd_change, names='index')

else:
    print('Problema ao realizar o login')

## Cadastrar movimentação diária
Em construção

In [225]:
library_path = "/sigaa/biblioteca/index.jsf"

library_page = session.get(urlparser.urljoin(base_url, library_path))

In [ ]:
post_path = (bs4.BeautifulSoup(library_page.text)
    .find('a', string='Cadastrar Movimentação Diária de Usuários')
    .find_parent('form')['action'])

post_data = { tag['name']: tag['value'] if tag.has_attr('value') else ""
    for tag in (bs4.BeautifulSoup(library_page.text)
        .find('a', string='Cadastrar Movimentação Diária de Usuários')
        .find_parent('form')
        .find_all('input')
    )
}

post_data['form:j_id_jsp_1241615306_121'] = 'form:j_id_jsp_1241615306_121' # TODO: get by anchor <a> onclick script

# bs4.BeautifulSoup(library_page.text).find('a', string='Cadastrar Movimentação Diária de Usuários')['onclick']
lib_month_year_page = session.post(urlparser.urljoin(base_url, post_path), data=post_data)

In [ ]:
import IPython
IPython.display.HTML(lib_month_year_page.text)